# Task 4.4.0
Implement the DSPy Module: Create a DSPy module that uses the strategy you devise to generate a cooperative answer.

# DSPy pipeline with in the Multi-Hop fashion (in the forward). 

Steps:
1) Define the first step and second step signatures
2) Define the retriever function. consider dividing to 2 parts , and look for the docs in the most relevant folder. use ColBERTv2?
3) Define the dspy pipeline. use CoT?
4) Load the dataset
5) Unpack it into DSPy.examples
6) run the pipeline


DSPy signature 1: 

- input fields:
- The previous turns (question, answer) 
- the context retrieved by the retriever (the context should be retrieved using RAG.)
- the current question

output fields:
- previous turns
- context
- current question
- summary of the student's goal or interests
- pragmatic or cooperative need underlying the student's current question (based on the past conversation and retrieved spans)


Step 2 dspy signtures

- input fields - > output from step 1.
- output field: a cooperative answer to the current question.


# Design plan

1) Define DSPy Signatures
Step 1 (Interpretation): history, current Q, context → history, current Q, context, student_goal, pragmatic_need
Step 2 (Retrieval): expanded_query, conversation_summary → refined_context
Step 3 (Answering): history, current Q, summary, goal, need, refined_context → cooperative_answer

2) Implement Retriever
Index HTML sources.
Retrieval cascade: BM25 (topic folder) → ColBERTv2/FAISS (fine passages).

3) Build Pipeline Forward Pass
Step 1 → Step 2 → Step 3.
Add loop if retrieval weak (multi-hop).

4) Prepare Data
Load PragmatiCQA train/val/test JSONL.
Convert each round to DSPy example.
Truncate long histories.

5) Train & Run
Few-shot or supervised training on train set.
Run pipeline on val/test sets.

In [ ]:
# Configure the LLM Model

# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
import dspy
import os
from dotenv import load_dotenv
load_dotenv("../grok_key.ini")
lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
dspy.configure(lm=lm)

In [ ]:

class enriched_query(dspy.Signature):
    history = dspy.InputField()
    current_question = dspy.InputField()
    context = dspy.InputField()

    # Outputs: keep inputs
    history = dspy.OutputField()              # pass-through
    current_question = dspy.OutputField()     # pass-through
    context = dspy.OutputField()      # pass-through

    # enrichments
    student_goal = dspy.OutputField()
    pragmatic_need = dspy.OutputField()


class answer(dspy.Signature):
    history = dspy.InputField()
    current_question = dspy.InputField()
    context = dspy.InputField()
    student_goal = dspy.InputField()
    pragmatic_need = dspy.InputField()

    #
    cooperative_answer = dspy.OutputField()

In [12]:
import bm25s
import Stemmer
import os

# Original folder names
folder_names = os.listdir("../PragmatiCQA-sources")

stemmer = Stemmer.Stemmer("english")

# Tokenize each folder name
corpus_tokens = bm25s.tokenize(folder_names, stopwords="en", stemmer=stemmer)

# Keep track of positions
retriever = bm25s.BM25(k1=0.9, b=0.4)
retriever.index(corpus_tokens)

def search(query: str, k: int):
    tokens = bm25s.tokenize(query, stopwords="en", stemmer=stemmer, show_progress=False)
    results, scores = retriever.retrieve(tokens, k=k, n_threads=1, show_progress=False)
    
    # 'results' is a list of token lists, so we can match them to the original folder names
    for i, token_list in enumerate(results):
        for j, ct in enumerate(corpus_tokens):
            if ct == token_list:
                folder_name = folder_names[j]
                break
        else:
            folder_name = "UNKNOWN"
        print(f"result: {folder_name} , score: {scores[i]}")

search("Harry", 3)


Split strings:   0%|          | 0/74 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/74 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/74 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/74 [00:00<?, ?it/s]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (74,) + inhomogeneous part.